In [1]:
## Tokenizer

In [2]:
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [36]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')

In [12]:
print(tokenizer.encode('이순신은 조선 중기의 무신이다.'))
#print(tokenizer.encode('이순신은 조선 중기의 무신이다.'))
print(tokenizer.decode([2, 10661, 2073, 3957, 9652, 2079, 15749, 28674, 18, 3]))

[2, 10661, 2073, 3957, 9652, 2079, 15749, 28674, 18, 3]
[CLS] 이순신은 조선 중기의 무신이다. [SEP]


In [37]:
# tokenizer.add_special_tokens("[TEST]")
# tokenizer.add_special_tokens("[/TEST]")

special_tokens_dict = {'additional_special_tokens': ['[TEST]','[C3]','[C4]']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
#special_tokens_dict = {'additional_special_tokens': config.new_special_token_list}
#num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

In [40]:
print(tokenizer.encode('이순신은 [TEST]조선[/TEST] 중기의 [C3]무신[C3]이다.'))

[2, 10661, 2073, 32000, 3957, 63, 19, 26967, 64, 9652, 2079, 32001, 15749, 32001, 30651, 18, 3]


In [38]:
tokenizer

PreTrainedTokenizerFast(name_or_path='klue/bert-base', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['[TEST]', '[C3]', '[C4]']})

[CLS] 이순신은 [TEST] 조선 [ / TEST ] 중기의 [C3] 무신 [C3] 이다. [SEP]


In [66]:
sentence_list = ['이순신은 PER 이순신이다 PER']

tokenized_sentences = tokenizer(
    sentence_list,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=256,
    add_special_tokens=True,
    #return_token_type_ids=False, # 문장 id
)

In [67]:
print(tokenizer.decode(tokenized_sentences.input_ids[0]))
tokenized_sentences.input_ids[0]

[CLS] 이순신은 PER 이순신이다 PER [SEP]


tensor([    2, 10661,  2073, 21639, 10661, 28674, 21639,     3])

In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split

dataset = pd.read_csv('../../../dataset/train/train.csv')
train_dataset, valid_dataset = train_test_split(dataset, test_size=0.2, stratify=dataset['label'], random_state=42)

In [71]:
valid_dataset.head()

,id,sentence,subject_entity,object_entity,label,source
12461,12461,영광군(군수 김준성)은 지난 18일 영광소방서와 합동으로 전라남도 문화재자료 제24...,"{'word': '영광군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '전라남도', 'start_idx': 32, 'end_idx': 3...",no_relation,wikitree
6520,6520,베이징올림픽에서는 국제야구연맹(IBAF)이 경기시간 단축을 위해 10회 말까지 승부...,"{'word': '국제야구연맹', 'start_idx': 10, 'end_idx':...","{'word': 'IBAF', 'start_idx': 17, 'end_idx': 2...",org:alternate_names,wikipedia
4584,4584,"가수 볼빨간사춘기 멤버 안지영, 우지윤 씨가 음원 사재기를 하는 사람들에게 일침 했다.","{'word': '볼빨간사춘기', 'start_idx': 3, 'end_idx': ...","{'word': '안지영', 'start_idx': 13, 'end_idx': 15...",per:employee_of,wikitree
15105,15105,메가박스중앙 주식회사는 중앙그룹 계열사이자 대한민국의 멀티플렉스 영화관 체인이다.,"{'word': '메가박스', 'start_idx': 0, 'end_idx': 3,...","{'word': '중앙그룹', 'start_idx': 13, 'end_idx': 1...",org:member_of,wikipedia
19336,19336,지난 25일(이하 현지 시각) 페이스북 CEO 마크 저커버그는 캘리포니아 산호세에서...,"{'word': '페이스북', 'start_idx': 17, 'end_idx': 2...","{'word': '마크 저커버그', 'start_idx': 26, 'end_idx'...",org:top_members/employees,wikitree


In [72]:
valid_dataset.label.value_counts()

no_relation                            1907
org:top_members/employees               857
per:employee_of                         714
per:title                               420
org:member_of                           373
org:alternate_names                     264
per:origin                              247
org:place_of_headquarters               239
per:date_of_birth                       226
per:alternate_names                     200
per:spouse                              159
per:colleagues                          107
per:parents                             104
org:founded                              90
per:date_of_death                        84
org:members                              84
org:product                              76
per:children                             61
per:place_of_residence                   39
per:other_family                         38
per:place_of_birth                       33
org:founded_by                           31
per:product                     

In [73]:
train_dataset.label.value_counts()

no_relation                            7627
org:top_members/employees              3427
per:employee_of                        2859
per:title                              1683
org:member_of                          1493
org:alternate_names                    1056
per:origin                              987
org:place_of_headquarters               956
per:date_of_birth                       904
per:alternate_names                     801
per:spouse                              636
per:colleagues                          427
per:parents                             416
org:founded                             360
org:members                             336
per:date_of_death                       334
org:product                             304
per:children                            243
per:place_of_residence                  154
per:other_family                        152
per:place_of_birth                      133
org:founded_by                          124
per:product                     

In [82]:
import json
from collections import defaultdict

dev_json_path = "../../../dataset/train/klue-re.json"

def json_to_df(json_path):
    with open(json_path) as f:
        json_object = json.load(f)
    data = defaultdict(list)
    for dict in json_object:
        for key, value in dict.items():
            data[key].append(value) 

    df = pd.DataFrame(data)
#     for key in df["subject_entity"][0].keys():
#         df["subject_" + key] =  df["subject_entity"].apply(lambda x : x[key])
#         df["object_" + key] =  df["object_entity"].apply(lambda x : x[key])  

    return df

In [83]:
json_to_df(dev_json_path).head()

,guid,sentence,subject_entity,object_entity,label,source
0,klue-re-v1_dev_00000,20대 남성 A(26)씨가 아버지 치료비를 위해 B(30)씨가 모아둔 돈을 훔쳐 인...,"{'word': 'A', 'start_idx': 7, 'end_idx': 7, 't...","{'word': '30', 'start_idx': 29, 'end_idx': 30,...",no_relation,wikitree
1,klue-re-v1_dev_00001,그러나 심 의원은 보좌진이 접속 권한을 받아 정부 업무추진비 사용 내역 등을 다운받...,"{'word': '심 의원', 'start_idx': 4, 'end_idx': 7,...","{'word': '정부', 'start_idx': 25, 'end_idx': 26,...",no_relation,wikipedia
2,klue-re-v1_dev_00002,"영화 《룸》에서 감금되어 살아가는 여자의 아들 '잭' 역으로 주목받으며, 크리틱스 ...","{'word': '잭', 'start_idx': 27, 'end_idx': 27, ...","{'word': '캐나다 스크린 어워드', 'start_idx': 65, 'end_...",no_relation,wikipedia
3,klue-re-v1_dev_00003,전라남도(도지사 김영록)는 해양수산부의 2020년 어촌뉴딜300 공모사업에 15개 ...,"{'word': '해양수산부', 'start_idx': 15, 'end_idx': ...","{'word': '도지사', 'start_idx': 5, 'end_idx': 7, ...",no_relation,wikitree
4,klue-re-v1_dev_00004,"부산항만공사(BPA, 사장 남기찬)는 27일 아동의 인권존중과 아동학대 예방에 대한...","{'word': '부산항만공사', 'start_idx': 0, 'end_idx': ...","{'word': '27일', 'start_idx': 21, 'end_idx': 23...",no_relation,wikitree


In [88]:
"2".rjust(3, "0")

'002'

In [92]:
#'====== ' + "Config loading... " + '='*10
#'test'.rjust(5, "=")
print(f'========== Config loading... '.ljust(50, "="))

test==============================================
